In [1]:
import mmap
import os
import random
import numpy as np
import time
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures

In [2]:
def get_time(fun):
    start_time = time.clock()
    fun()
    final_time = time.clock() - start_time
    return str(final_time) + " seconds"

### Генерация массива из целых 32х разрядных беззнаковых чисел

In [3]:
#length = 100000000
f_name_test = 'data_test.txt'
f_name = 'data.txt'

In [4]:
def write(fn):
    with open(fn, 'wb') as f:
        arr = np.random.randint(2, length, size=length//4, dtype = np.dtype('uint32')).newbyteorder('B').byteswap()                        
        f.write(arr.data)
        f.close()

In [5]:
length = 10000
write(f_name_test)

### Последовательные вычисления

In [5]:
def sequential_method():
    with open(f_name, 'r+b') as f:
        buf = f.read()
        data = np.frombuffer(buf, dtype=np.dtype('uint32').newbyteorder('B'))
        min_el, max_el, final_sum = make_calculation(data)
        f.close()

In [6]:
def make_calculation(data):

    global min_el
    global max_el
    global final_sum
    i = 0
    for num in data:
        if num < min_el:
            min_el = num
        if num > max_el:
            max_el = num
        final_sum += num
        i+=1
    print("min element:",min_el)
    print("max element:",max_el)
    print("sum of elements:",final_sum)
    return [min_el, max_el, final_sum]

### MMF

In [19]:
def calculations(mm,l):   
    
    global final_sum
    global max_el
    global min_el
    
    for i in range(0, l):
        arr = mm[i*32: (i+1)*32]
        for el in arr:
            if el > max_el:
                max_el = el
            if el < min_el:
                min_el = el                    
            final_sum += el

In [18]:
def thread_calculations(mm,l):   
    
    finalSum = 0
    global max_el
    global min_el
    
    for i in range(0, l):
        arr = mm[i*32: (i+1)*32]
        for el in arr:
            if el > max_el:
                max_el = el
            if el < min_el:
                min_el = el                    
            finalSum += el
    #print(finalsum)
    return finalSum

In [8]:
def make_calculation_mmf():
    try:  
         with open(f_name, 'r+b') as f:
                buf = mmap.mmap(f.fileno(), length=length, offset=0, access=mmap.ACCESS_READ)
                array = np.frombuffer(buf, dtype=np.dtype('uint32').newbyteorder('B'))
                calculations(array,length)
    finally:
        print("min element:",min_el)
        print("max element:",max_el)
        print("sum of elements:",final_sum)

### Threads + MMF

In [9]:
import threading

In [20]:
def make_threads(count, mm, el_count):
    l = el_count//count
    indexA = 0 
    indexB = l
    i = 1;
  #not interesting 
    #for thread in range(0,count):
     #   if thread == count-1:
      #      threads.append(threading.Thread(target=calculations, args=(mm[indexA:], length)))
       # else:
        #    threads.append(threading.Thread(target=calculations, args=(mm[indexA:indexB], length)))
        #indexA = i * length;
        #i+=1
        #indexB = i * length;
    
   # for thread in threads:
    #    thread.start()
    #for thread in threads:
     #   thread.join()
    
    #interesting
    executor = ThreadPoolExecutor(max_workers=count)
    futures = []
    for future_num in range(0,count):
        if future_num == count-1:
            futures.append(executor.submit(thread_calculations, mm[indexA:], l))
        else:
            futures.append(executor.submit(thread_calculations, mm[indexA:indexB], l))
        indexA = i * l;
        i+=1
        indexB = i * l;
    return futures

In [23]:
def make_calculation_mmf_threads():
    final_sum = 0
    futures = []
    try:
        with open(f_name, 'r+b') as f:
                buff = mmap.mmap(f.fileno(), length=length, offset=0, access=mmap.ACCESS_READ)
                array = np.frombuffer(buff, dtype=np.dtype('uint32').newbyteorder('B'))
                futures = make_threads(threads_count, array, length)
    finally:
        results = [fut.result() for fut in concurrent.futures.as_completed(futures)]
        print("min element:",min_el)
        print("max element:",max_el)
        print("sum of elements:",sum(results))

#### Тестирование на небольших данных

In [29]:
threads_count = 10
f_name = 'data_test.txt'

In [30]:
length = 100000
write(f_name)

max_el = final_sum = 0
min_el = 10**32
print('Последовательный метод:',get_time(sequential_method))
max_el = final_sum = 0
min_el = 10**32
print('С использованием MMF:',get_time(make_calculation_mmf))
max_el = final_sum = 0
min_el = 10**32
print('С использованием потоков + MMF:',get_time(make_calculation_mmf_threads))

min element: 11
max element: 99997
sum of elements: 1244474330
Последовательный метод: 0.023244699999963814 seconds
min element: 11
max element: 99997
sum of elements: 1244474330
С использованием MMF: 0.22133629999996174 seconds
min element: 11
max element: 99997
sum of elements: 1244474330
С использованием потоков + MMF: 0.24804250000011052 seconds


In [32]:
length = 1000000
write(f_name) 

max_el = final_sum = 0
min_el = 10**32
print('Последовательный метод:',get_time(sequential_method))
max_el = final_sum = 0
min_el = 10**32
print('С использованием MMF:',get_time(make_calculation_mmf))
max_el = final_sum = 0
min_el = 10**32
print('С использованием потоков + MMF:',get_time(make_calculation_mmf_threads))

min element: 3
max element: 999999
sum of elements: 124896822844
Последовательный метод: 0.2320250999999871 seconds
min element: 3
max element: 999999
sum of elements: 124896822844
С использованием MMF: 2.4709411000000046 seconds
min element: 3
max element: 999999
sum of elements: 124896822844
С использованием потоков + MMF: 2.4772308000000294 seconds


### Тестирование на файле размера 1 ГБ

In [24]:
length = 90000000
f_name = 'data.txt'

In [25]:
write(f_name)

In [26]:
max_el = final_sum = 0
min_el = 10**32
print('Последовательный метод:',get_time(sequential_method))

min element: 17
max element: 89999998
sum of elements: 1012430240809726
Последовательный метод: 21.508851999999962 seconds


In [27]:
max_el = final_sum = 0
min_el = 10**32
print('С использованием MMF:',get_time(make_calculation_mmf))

min element: 17
max element: 89999998
sum of elements: 1012430240809726
С использованием MMF: 229.6234756 seconds


In [28]:
max_el = final_sum = 0
min_el = 10**32
print('С использованием потоков + MMF:',get_time(make_calculation_mmf_threads))

min element: 17
max element: 89999998
sum of elements: 1012430240809726
С использованием потоков + MMF: 232.3467584 seconds
